In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
from nltk.corpus import stopwords
stopwords=stopwords.words("english")

In [4]:
from nltk.stem import PorterStemmer

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
from keras.preprocessing import sequence

/home/silverstar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.recurrent import SimpleRNN
from keras.layers import concatenate,Flatten,Reshape

In [11]:
from keras.models import Model

In [12]:
from keras.layers import RNN
from keras.layers import Input

In [13]:
import keras.backend as K

In [14]:
df_atnt=pd.read_csv("chdata/att")
df_verizon=pd.read_csv("chdata/verizon")
df_t=pd.read_csv("chdata/t-mobile")

df_label=pd.concat([df_atnt,df_verizon,df_t],axis=0)

df_label.columns

df_label=df_label[['brand','tid','choose_one','choose_one:confidence']]

df_label.columns=['brand','id','choose_one',"choose_one:confidence"]
df_label=df_label.reindex()

In [15]:
df=pd.read_json("crawler-tid/tweets_json",lines=True)

In [16]:
mainDF=pd.merge(df,df_label,on="id")

In [17]:
mainDF.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'possibly_sensitive_appealable', 'retweet_count',
       'retweeted', 'source', 'text', 'truncated', 'user',
       'withheld_in_countries', 'brand', 'choose_one',
       'choose_one:confidence'],
      dtype='object')

In [18]:
mainDF=mainDF[['id','text','brand','choose_one']]

In [19]:
brands=np.unique(mainDF['brand'])

# Doing Pre-Processing

In [20]:
porter=PorterStemmer()
def preprocessing(a_tweet):
    cleanTweet=[]
    a_tweet=a_tweet.split(" ")
    for word in a_tweet:
        word=word.strip()
        valid=re.search(pattern=r"[^A-Za-z]",string=word)==None
        if(word!="" and valid==True and word not in stopwords):
            word=porter.stem(word)
            cleanTweet.append(word)
    return " ".join(cleanTweet)

In [21]:
mainDF['text']=mainDF['text'].apply(preprocessing)

mainDF=mainDF.dropna()

In [22]:
mainDF.shape

(3930, 4)

In [23]:
countVect=CountVectorizer()
countVect.fit_transform(df['text'])

<3930x8211 sparse matrix of type '<class 'numpy.int64'>'
	with 60211 stored elements in Compressed Sparse Row format>

In [24]:
vocab=countVect.get_feature_names()

In [66]:
vocab[3571]

'if'

In [25]:
def convertWord(tweet):
    tweet=tweet.split(" ")
    indexTweet=[]
    for word in tweet:
        word=word.lower()
        if word in vocab:
            indexTweet.append(vocab.index(word))
    return indexTweet

In [26]:
mainDF['text']=mainDF['text'].apply(convertWord)

In [27]:
max_tweet_length=-1
max_tweet_index=0
for tweet in df['text']:
    if(len(tweet)>max_tweet_length):
        max_tweet_length=len(tweet)
        max_tweet_index=tweet

In [28]:
mainDF['brand']=LabelEncoder().fit_transform(mainDF['brand'])
mainDF['label']=LabelEncoder().fit_transform(mainDF['choose_one'])

In [29]:
X=sequence.pad_sequences(mainDF['text'].ravel(),max_tweet_length)
y=mainDF['label']

In [30]:
X=pd.DataFrame(X)

In [31]:
X['brand']=mainDF['brand']

In [32]:
X.shape

(3930, 157)

In [35]:
X_train,X_test,y_train,y_test=train_test_split(X,y.ravel(),stratify=y.ravel())

In [36]:
brand_train=X_train['brand']
X_train=X_train.drop(['brand'],axis=1)
brand_test=X_test['brand']
X_test=X_test.drop(['brand'],axis=1)

In [40]:
X_train=X_train.values
X_test=X_test.values

In [41]:
top_words=len(vocab)
embedding_vecor_length = 32

In [42]:
max_tweet_length

156

In [46]:
embedding_vecor_length

32

In [87]:
firstInput = Input(shape=(max_tweet_length,),name="embInput")
firstEmbedding=Embedding(output_dim=embedding_vecor_length, input_dim=len(vocab))(firstInput)
embeddingFlatten=Flatten()(firstEmbedding)

In [88]:
secondInput=Input(shape=(1,),name="brandInput")
secondDense=Dense(1,)(secondInput)

In [89]:
concat=concatenate([embeddingFlatten,secondInput],axis=1)
reshape=Reshape((embedding_vecor_length*max_tweet_length+1,1))(concat)
lstm=LSTM(units=3)(reshape)
dense=Dense(1,activation="sigmoid",name="output")(concat)

In [90]:
model=Model(inputs=[firstInput,secondInput],outputs=dense)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [91]:
print(X_train.shape)
print(brand_train.shape)

(2947, 156)
(2947,)


In [92]:
model.fit({"embInput":X_train,"brandInput":brand_train},{"output":y_train},
          validation_data=[{"embInput":X_train,"brandInput":brand_train},{"output":y_train}],
          epochs=20,batch_size=128)

Train on 2947 samples, validate on 2947 samples
Epoch 1/20
2947/2947 [==============================] - 1s 189us/step - loss: 0.5482 - acc: 0.7747 - val_loss: 0.5134 - val_acc: 0.7896
Epoch 2/20
2947/2947 [==============================] - 0s 86us/step - loss: 0.5130 - acc: 0.7896 - val_loss: 0.5106 - val_acc: 0.7896
Epoch 3/20
2947/2947 [==============================] - 0s 87us/step - loss: 0.5110 - acc: 0.7896 - val_loss: 0.5048 - val_acc: 0.7896
Epoch 4/20
2947/2947 [==============================] - 0s 85us/step - loss: 0.5009 - acc: 0.7896 - val_loss: 0.4930 - val_acc: 0.7896
Epoch 5/20
2947/2947 [==============================] - 0s 87us/step - loss: 0.4888 - acc: 0.7896 - val_loss: 0.4737 - val_acc: 0.7896
Epoch 6/20
2947/2947 [==============================] - 0s 83us/step - loss: 0.4658 - acc: 0.7896 - val_loss: 0.4498 - val_acc: 0.7896
Epoch 7/20
2947/2947 [==============================] - 0s 86us/step - loss: 0.4380 - acc: 0.7896 - val_loss: 0.4159 - val_acc: 0.7903
Epoch 

In [104]:
y_prob=model.predict({"embInput":X_test,"brandInput":brand_test})

In [105]:
y_prob

array([[0.1252428 ],
       [0.9010598 ],
       [0.99343216],
       [0.4181793 ],
       [0.80232626],
       [0.77062374],
       [0.81931007],
       [0.9965148 ],
       [0.8685977 ],
       [0.8777976 ],
       [0.70923865],
       [0.9853495 ],
       [0.81318563],
       [0.56109095],
       [0.95277596],
       [0.90179443],
       [0.85051066],
       [0.9653162 ],
       [0.8858631 ],
       [0.99149126],
       [0.85528904],
       [0.9603406 ],
       [0.62666726],
       [0.92308146],
       [0.64857453],
       [0.4366684 ],
       [0.91617376],
       [0.42922217],
       [0.88509685],
       [0.96638685],
       [0.8511429 ],
       [0.85494924],
       [0.5945411 ],
       [0.9289138 ],
       [0.50839114],
       [0.857292  ],
       [0.98630685],
       [0.928534  ],
       [0.9654588 ],
       [0.93580234],
       [0.6963478 ],
       [0.95082265],
       [0.7229002 ],
       [0.90074176],
       [0.8560864 ],
       [0.9711882 ],
       [0.91929287],
       [0.782

In [102]:
y_pred[y_pred>0.5]=1

In [103]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [97]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [83]:
y_pred=np.argmax(y_prob,axis=1)

In [85]:
np.mean(y_pred==y_test)

0.2105798575788403

In [78]:
np.mean(y_pred==y_test)

0.0

In [75]:
print(classification_report(y_test,y_pred,target_names=['class 0','class 1']))
print(confusion_matrix(y_test,y_pred))
print("Accuracy="+str(np.mean(y_test==y_pred)))

              precision    recall  f1-score   support

     class 0       0.42      0.69      0.53       207
     class 1       0.90      0.75      0.82       773

   micro avg       0.74      0.74      0.74       980
   macro avg       0.66      0.72      0.67       980
weighted avg       0.80      0.74      0.76       980

[[143  64]
 [194 579]]
Accuracy=0.5901686797167847
